# 文本分类提取到标签中
利用大模型得function call 能力将文本中提取到得标签注入到实体中

## 加载环境变量

In [1]:
import os

from dotenv import find_dotenv, load_dotenv

# 读取本地/项目的环境变量。
# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
env = load_dotenv(find_dotenv())
os.environ["LANGCHAIN_API_KEY"]

'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'

## 定义一个实体描述所需要得属性

In [8]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

tagging_prompt = ChatPromptTemplate.from_template(
    """从以下段落中提取所需要得信息，仅提取'{class_name}' 类实体中提到的属性。
    
    段落：
    {input}
    """
)


class Classification(BaseModel):
    sentiment: str = Field(description="文本的情绪")
    aggressiveness: int = Field(
        description="文本有多激进，范围在1到10内"
    )
    language: str = Field(description="文本所使用的语言，中文，英文，日语等")


# LLM
base_url = os.environ["BASE_URL"]
api_key = os.environ["API_KEY"]
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key, base_url=base_url).with_structured_output(Classification)

## 调用大模型返回实体

In [11]:
input_text = "今天遇到了一个傻波依，差点没被气死掉。"
prompt = tagging_prompt.invoke({"class_name": "Classification", "input": input_text})
response = llm.invoke(prompt)

response

Classification(sentiment='负面', aggressiveness=7, language='中文')

## 返回类

In [12]:
input_text = "今天阳光明媚，心情很好。"
prompt = tagging_prompt.invoke({"class_name": "Classification", "input": input_text})
response = llm.invoke(prompt)

response.dict()

{'sentiment': '积极', 'aggressiveness': 1, 'language': '中文'}